##### Task (20 Points)

In [ ]:
""" 
    topic: search algorithms

    a)
    implement BFS (Breadth-first Search)

    b)
    implement DFS (Depth-first Search)

    c)
    implement a function that calculates all connected components
"""

In [65]:
def BFS(Graph, startknoten):
    R = [startknoten]
    Q = [startknoten]
    F = []
    while len(Q) != 0:
        nachbarslistevonKnotenausQ = Graph.get_node(Q[0]).adjacent_nodes()
        fertig = True
        for nachbar in nachbarslistevonKnotenausQ:
            if nachbar.id() not in R:
                R.append(nachbar.id())
                Q.append(nachbar.id())
                F.append((Q[0], Q[-1]))
                fertig = False
                break
        if fertig == True:
            Q.pop(0)
        #print(len(Q))
    return R, F

In [56]:
def DFS(Graph, startknoten):
    R = [startknoten]
    Q = [startknoten]
    F = []
    while len(Q) != 0:
        nachbarslistevonKnotenausQ = Graph.get_node(Q[-1]).adjacent_nodes()
        fertig = True
        for nachbar in nachbarslistevonKnotenausQ:
            if nachbar.id() not in R:
                R.append(nachbar.id())
                Q.append(nachbar.id())
                F.append((Q[-2], Q[-1]))
                fertig = False
                break
        if fertig == True:
            Q.pop()
        #print(len(Q))

    return R, F

In [54]:
import numpy as np


class Graph:
    invalid_node = -1

    def __init__(self, numorfile, directed=False):
        self._directed = directed
        if type(numorfile) == int:
            self._nodes = [self.Node() for i in range(numorfile)]
        elif type(numorfile) == str:
            with open(numorfile, "r") as file:
                numnodes = int(file.readline())
                self._nodes = [self.Node() for i in range(numnodes)]
                for line in file:
                    tail = int(line.split()[0])
                    head = int(line.split()[1])
                    if tail != head:
                        self.add_edge(tail, head)
                    else:
                        raise RuntimeError("Invalid file format: loops not allowed")
        else:
            raise NotImplementedError("Type of Argument not allowed")

    def num_nodes(self):
        return len(self._nodes)

    def add_nodes(self, num_new_nodes):
        self._nodes.extend([self.Node() for i in range(num_new_nodes)])

    def add_edge(self, tail, head):
        if tail >= self.num_nodes() or tail < 0 or head >= self.num_nodes() or head < 0:
            raise ValueError("Edge cannot be added due to undefined endpoint")
        self._nodes[tail].add_neighbor(head)
        if not self._directed:
            self._nodes[head].add_neighbor(tail)

    def get_node(self, nodeId):
        if nodeId < 0 or nodeId >= self.num_nodes():
            raise ValueError("Invalid nodeId")
        return self._nodes[nodeId]

    def __str__(self):  # for printing
        out = ""
        if self._directed:
            out += "Digraph "
        else:
            out += "Undirected Graph "
        out += "with {} vertices, numbered 0, ..., {}\n".format(
            self.num_nodes(), self.num_nodes() - 1
        )
        for nodeId in range(self.num_nodes()):
            if self._directed:
                s = "leaving"
            else:
                s = "incident to"
            out += "The following edges are " + s + " vertex {}:\n".format(nodeId)
            for neighbor in self._nodes[nodeId].adjacent_nodes():
                out += "{} - {}\n".format(nodeId, neighbor.id())
        return out

    def is_simple(self):
        for nodeId in range(self.num_nodes()):
            nachbarsliste = []
            for neighbor in self._nodes[nodeId].adjacent_nodes():
                nachbarsliste.append(neighbor.id())

            unique_nachbarsliste = []
            for nachbar in nachbarsliste:
                if nachbar not in unique_nachbarsliste:
                    unique_nachbarsliste.append(nachbar)
                else:
                    return False

        return True
    
    def components(self):
        components = []
        visited_nodes = set()

        for nodeId in range(self.num_nodes()):
            if nodeId not in visited_nodes:
                start_node = nodeId
                nodes, _ = DFS(self, start_node)
                components.append(nodes)
                visited_nodes.update(nodes)

        return components

    def adjancency_matrix(self):
        num_nodes = self.num_nodes()
        matrix = [[0] * num_nodes for _ in range(num_nodes)]

        for nodeId in range(num_nodes):
            neighbors = self._nodes[nodeId].adjacent_nodes()
            for neighbor in neighbors:
                matrix[nodeId][neighbor.id()] = 1
        return np.array(matrix)

    class Node:
        def __init__(self):
            self._neighbors = []

        def add_neighbor(self, nodeId):
            self._neighbors.append(Graph.Neighbor(nodeId))

        def adjacent_nodes(self):
            return self._neighbors

        def number_of_neighbors(self):
            return len(self._neighbors)

    class Neighbor:
        def __init__(self, nodeId):
            self._id = nodeId

        def id(self):
            return self._id

file = "./graph.txt"
g = Graph(file, directed=True)

In [58]:
print(g)

Digraph with 6 vertices, numbered 0, ..., 5
The following edges are leaving vertex 0:
0 - 1
The following edges are leaving vertex 1:
1 - 5
The following edges are leaving vertex 2:
The following edges are leaving vertex 3:
3 - 4
The following edges are leaving vertex 4:
4 - 0
The following edges are leaving vertex 5:
5 - 0



In [63]:
g.components()

[[0, 1, 5], [2], [3, 4, 0, 1, 5]]

In [66]:
print(DFS(g, 0))
print(BFS(g, 0))

([0, 1, 5], [(0, 1), (1, 5)])
([0, 1, 5], [(0, 1), (1, 5)])
